# PyTorch Tutorial: Practical Example - Image Classification

Classifying handwritten digits using MNIST dataset.

## Learning Objectives
- Work with image data
- Build CNNs
- Train classification models
- Evaluate performance

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)
print("PyTorch version:", torch.__version__)

## Load MNIST Dataset

In [ ]:
# Transform: convert to tensor and normalize
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load datasets
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Image shape: {train_dataset[0][0].shape}")

## Visualize Data

In [ ]:
# Show some examples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i in range(10):
    img, label = train_dataset[i]
    axes[i//5, i%5].imshow(img.squeeze(), cmap="gray")
    axes[i//5, i%5].set_title(f"Label: {label}")
    axes[i//5, i%5].axis("off")
plt.tight_layout()
plt.show()

## Build CNN Model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 28x28 -> 14x14
        x = self.pool(torch.relu(self.conv2(x)))  # 14x14 -> 7x7
        x = x.view(-1, 32 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNN()
print(model)

## Train the Model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
train_losses = []

print("Training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

print("\nTraining complete!")

## Evaluate Model

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, targets in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Correct: {correct}/{total}")

## Visualize Predictions

In [ ]:
# Get some test samples
model.eval()
with torch.no_grad():
    data, targets = next(iter(test_loader))
    outputs = model(data)
    _, predicted = torch.max(outputs, 1)

# Plot
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i in range(10):
    axes[i//5, i%5].imshow(data[i].squeeze(), cmap="gray")
    color = "green" if predicted[i] == targets[i] else "red"
    axes[i//5, i%5].set_title(f"Pred: {predicted[i]}, True: {targets[i]}", color=color)
    axes[i//5, i%5].axis("off")
plt.tight_layout()
plt.show()

## Key Takeaways

1. CNNs are great for image data
2. CrossEntropyLoss for classification
3. Data loaders handle batching
4. Accuracy metric for classification

**Excellent! You completed image classification! 🎉**